<h1><center>Exploring Graduation Rates By Ethnicity</center></h1>

**By: Robert Campbell**

In [93]:
import cbell as cb

In [187]:
import pandas as pd
import numpy as np

#conda install -c conda-forge cufflinks-py
#conda install plotly
import ipywidgets as wg
from IPython.display import display
import pandas as pd 
import numpy as np
import cufflinks as cf
import chart_studio.plotly as py

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

%matplotlib inline
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import inspect
import seaborn as sns
init_notebook_mode(connected=True)
cf.go_offline()

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
pd.options.display.max_columns = 200
pd.options.display.max_rows = 272

from joblib import dump, load
from Logger import RegressionLogger, FuncTransformer, ColSelect

In [3]:
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")
df.drop("Unnamed: 0",axis=1, inplace=True)
institution_name = df["Institution Name"]
df.drop(["Institution Name", "City location of institution (HD2019)"], axis=1, inplace=True)
X = df.iloc[:,:-14]
X = pd.get_dummies(X, drop_first=True)
X["Institution Name"] = institution_name

grad_rates = df.iloc[:,-14:]
X = pd.concat([X, grad_rates], axis=1)

In [4]:
X_df = load("objects/engineered_features.joblib")
y = load("objects/y.joblib")
x_train, x_test, y_train, y_test, x_holdout, y_holdout = load("objects/holdout_total.joblib")

In [5]:
model_log = load("objects/model_logging.joblib")

In [6]:
# from joblib import dump, load
model = model_log[4]['model']

In [7]:
# load in feature engineered features
X_fe = load("objects/features.joblib")

In [8]:
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")

In [9]:
means = df.filter(regex='Grad').describe().loc['mean'].sort_values().to_numpy()
cols = ['Black', 'American Indian', 'Hispanic', 'men', 'two or more races', 
        'Native Hawaiian', 'unknown', 'total cohort', 'White',
        'women', 'Asian/Native Hawaiian', 'Nonresident alien', 'Asian']

#cols = cols.str.replace("Graduation rate", "")
#cols = cols.str.replace("DRVGR2019", "")
#cols = cols.str.replace("Other Pacific Islander", "")

fig = go.Figure()
_ = fig.add_trace(go.Bar(
    x=cols, y=means, text=np.round(means, 2)
))
_ = fig.update_layout(
    title='Mean Graudation Rates',
    xaxis={
        'title': "Gender/Race"
    },
    yaxis = {
        'title': "Mean Graduation Rate"
    },
    height = 600
)

In [10]:
fig.show()

<center><h3>4 year institutions<h3></center>

<h3><center>2019 Cohort</center></h3>

<center><h3>6 year Graduation Rates</h3></center>

<h1><center>Problem Statement</center></h1>

## <ul><li>Problem</li></ul>
Duquesne University is facing pressure from the community claiming that the university is underperforming particularly with students of color. The University needs to respond to the community with a data backed explanation for their past performance along with a data driven plan to improve outcome measures (Graduation Rates)

## <ul><li>Source of Data</li></ul>

The Data Science team has access to the free publicly available Ipeds Data Center, which has data about all 4-year public and private institutions.

<a src='https://nces.ed.gov/ipeds/use-the-data'>Ipeds Data Center</a>

In [11]:
# Reset Our Data Frame
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")
df.drop("Unnamed: 0",axis=1, inplace=True)

<h1><center>Overview of Features</center></h1>

<h3><center>Numeric Features</center></h3>
<center><ul>
    <li>Revenues and Expenses</li>
    <li>Price of School</li>
    <li>Admissions Test Scores</li>
    <li>Staff Diversity</li>
    <li>Enrollment Diversity</li>
    <li>Student-to-Faculty Ratio</li>
</ul></center>

<h3><center>HBCU Graduation Rates Vs Non HBCU Schools</center></h3>
<ul>
    <li>State</li>
    <li>Sector</li>
    <li>Carnegie Classification</li>
    <li>HBCU</li>
</ul>

## University_lookup

In [177]:
index_loc = y_test.index[1]
df.loc[index_loc, "Institution Name"]

'Duquesne University'

In [100]:
regex = "Duques"
my_id = df[df["Institution Name"].str.contains(regex)].index[0]
sample = X_fe.loc[[my_id], :]
iloc = df.index.get_loc(my_id)
sample_y = df.iloc[[iloc],-14:]

In [126]:
sample_y

,Graduation rate total cohort (DRVGR2019),Graduation rate men (DRVGR2019),Graduation rate women (DRVGR2019),Graduation rate American Indian or Alaska Native (DRVGR2019),Graduation rate Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019),Graduation rate Asian (DRVGR2019),Graduation rate Native Hawaiian or Other Pacific Islander (DRVGR2019),Graduation rate Black non-Hispanic (DRVGR2019),Graduation rate Hispanic (DRVGR2019),Graduation rate White non-Hispanic (DRVGR2019),Graduation rate two or more races (DRVGR2019),Graduation rate Race/ethnicity unknown (DRVGR2019),Graduation rate Nonresident alien (DRVGR2019),Transfer-out rate total cohort (DRVGR2019)
UnitID,,,,,,,,,,,,,,
212106,81.0,77.0,83.0,100.0,60.0,57.0,100.0,56.0,79.0,83.0,71.0,77.0,50.0,NaN


<h1><center>Graduation Rates By Sector</center></h1>

In [12]:
sector = df.filter(regex="Sector").columns[0]
black = df.filter(regex="Grad.*Black").columns[0]
total = df.filter(regex="Grad").columns[0]

In [13]:
all_sectors = df[sector].unique()

sector_group = df.groupby(by=sector)

grad_rate_by_sector = sector_group.agg('mean').sort_values(by=black)[[black, total]]

In [14]:
grad_rate_by_sector.drop(['Public, 2-year', 'isMissing', 'Private not-for-profit, 2-year', 
                          'Private for-profit, 2-year', 'Private for-profit, less-than 2-year'],
                         inplace=True)

## BootStrap Confidence Interval for Black Graduation Rates Across Sector

In [15]:
sector_error1 = cb.get_error(sector_group.get_group(all_sectors[1])[black].dropna(), np.mean, 10000)
sector_error2 = cb.get_error(sector_group.get_group(all_sectors[2])[black].dropna(), np.mean, 10000)
sector_error3 = cb.get_error(sector_group.get_group(all_sectors[3])[black].dropna(), np.mean, 10000)
sector_error4 = cb.get_error(sector_group.get_group(all_sectors[1])[total].dropna(), np.mean, 10000)
sector_error5 = cb.get_error(sector_group.get_group(all_sectors[2])[total].dropna(), np.mean, 10000)
sector_error6 = cb.get_error(sector_group.get_group(all_sectors[3])[total].dropna(), np.mean, 10000)


diff1 = sector_group.get_group(all_sectors[1])[total] - sector_group.get_group(all_sectors[1])[black]
diff2 = sector_group.get_group(all_sectors[2])[total] - sector_group.get_group(all_sectors[2])[black]
diff3 = sector_group.get_group(all_sectors[3])[total] - sector_group.get_group(all_sectors[3])[black]

diff_error1 = cb.get_error(diff1.dropna(), np.mean, 10000)
diff_error2 = cb.get_error(diff2.dropna(), np.mean, 10000)
diff_error3 = cb.get_error(diff3.dropna(), np.mean, 10000)

In [16]:
grad_rate_by_sector["Mean_Difference"] = grad_rate_by_sector[total] - grad_rate_by_sector[black]

In [145]:
fig = make_subplots(2,1)

line = go.Scatter(
    y=[0, 1],x=[7, 7],mode= 'lines',
    showlegend=False, hoverinfo='none'
)

bar1 =  go.Bar(
    x=grad_rate_by_sector.index, y=grad_rate_by_sector[total], 
    text=np.round(grad_rate_by_sector[total], 1), name="Total Cohort",
    error_y=dict(type='data', array=[sector_error4, sector_error5, sector_error6]))

bar2 = go.Bar(
    x=grad_rate_by_sector.index, y=grad_rate_by_sector[black], 
    text=np.round(grad_rate_by_sector[black], 1), name="African American",
    error_y=dict(type='data', array=[sector_error1, sector_error2, sector_error3]),
)
fig.add_trace(bar1, 1, 1, secondary_y)
fig.add_trace(line, 1, 1, secondary_y=True)

_ = fig.update_layout(
    title="Mean Graduation Rates By Sector of Institution",
    xaxis={
        'title': "Sector"
    },
    yaxis={
        'title': 'Mean Graduation Rate'
    },
    font=dict(
        family="Courier New, monospace",
        size=16,
    )
)
fig.show()

This is the format of your plot grid:
[ (1,1) x,y,y2 ]



<h4><li><center>Difference Between African American Graduation Rates and Total Cohort Graduation</center></li></h4>

In [18]:
fig = go.Figure(
    data = [
        go.Bar(x=grad_rate_by_sector.index, y=grad_rate_by_sector["Mean_Difference"], 
               text=np.round(grad_rate_by_sector["Mean_Difference"], 1), name="Difference of Means",
               error_y=dict(type='data', array=[diff_error1, diff_error2, diff_error3])),
    ]
)

_ = fig.update_layout(
    title="Difference Between Black and Total Graduation Rates By Sector of Institution",
    xaxis={
        'title': "Difference By Sector"
    },
    yaxis={
        'title': 'Difference in Graduation Rate'
    },
    font=dict(
        family="Courier New, monospace",
        size=16,
    )
)
fig.show()

<h1><center>Graduation Rates By State</center></h1>

In [19]:
state = df.filter(regex="State").columns[0]
all_states = df[state].unique()

In [20]:
state_group = df.groupby(by=state)

In [21]:
grad_rate_by_state = state_group.agg('mean').sort_values(by=black)[[black, total]].dropna()

In [22]:
state_yblack = grad_rate_by_state[black].to_numpy()[0:51:10]
state_y = grad_rate_by_state[total].to_numpy()[0:51:10]

fig = go.Figure(
    data = [
        go.Bar(x=grad_rate_by_state.index[0:51:10], y=state_y, 
               text=np.round(state_y, 1), name="Total Cohort"),
        go.Bar(x=grad_rate_by_state.index[0:51:10], y= state_yblack, 
               text=np.round(state_yblack, 1), name="African American"),
        
    ]
)

_ = fig.update_layout(
    title="Mean Graduation Rates By State",
    xaxis={
        'title': "State"
    },
    yaxis={
        'title': 'Mean Graduation Rate'
    },
    font=dict(
        family="Courier New, monospace",
        size=16,      
    )
)
fig.show()     

<h1><center>HBCU Graduation Rates Vs Non HBCU Schools</center></h1>

In [23]:
hbcu = df.filter(regex="Historically").columns[0]
hbcu_group = df.groupby(by=hbcu)
grad_rate_hbcu = hbcu_group.agg('mean').sort_values(by=black)[[black, total]]

In [24]:
grad_rate_hbcu = grad_rate_hbcu.drop("isMissing")

In [25]:
fig = go.Figure(
    data = [
        go.Bar(x=grad_rate_hbcu.index, y=grad_rate_hbcu[total], 
               text=np.round(grad_rate_hbcu[total], 1), name="Total Cohort"),
        go.Bar(x=grad_rate_hbcu.index, y=grad_rate_hbcu[black], 
               text=np.round(grad_rate_hbcu[black], 1), name="African American"),
    ]
)

_ = fig.update_layout(
    title="Mean Graduation Rates For Historically Black Colleges and Universities",
    xaxis={
        'title': "Is_HBCU"
    },
    yaxis={
        'title': 'Mean Graduation Rate'
    },
    font=dict(
        family="Courier New, monospace",
        size=16,
    )
    
)
fig.show()

<h2><center>Whats Wrong With These comparisons?</center></h2>

<h3><ol><li>Correlation Between Features</li></ol></h3>

<h3><ol start=2><li>Controlling for other Features</li></ol></h3>

In [26]:
hbcu_df = df[df[hbcu] == 'Yes']
hwcu_df = df[df[hbcu] == 'No']
missing = df[df[hbcu] == 'isMissing']

In [27]:
price = df.filter(regex='price').columns[2]
test_score = df.filter(regex='Composite').columns[0]

<h1><center>HBCU Graduation Rates Vs Price</center></h1>

In [151]:
range_ = [12, 26]
degree = wg.IntSlider(value=1, min=1, max=3)

feature_picker = wg.Dropdown(
    options=X.columns,
    value=X.filter(regex="price").columns[-1],
    description='Feature:',
    disabled=False,
)
target_picker = wg.Dropdown(
    options=grad_rates.columns,
    value=grad_rates.columns[0],
    description='Target:',
    disabled=False,
)
color_picker = wg.Dropdown(
    options=X.filter(regex="Sector|Historically|State|Basic|Size and Setting|category").columns,
    value=X.filter(regex="Historically").columns[0],
    description='Color:',
    disabled=False,
)


def fe_plot(feature, target, color):
    
    fig2 = go.Figure()
    
    fig2.add_trace(go.Scatter(
        x=X[X[color] == 0.0][feature], y=X[X[color] == 0.0][target], mode="markers", name="No", hovertext=X["Institution Name"]
    ))
    
    fig2.add_trace(go.Scatter(
        x=X[X[color] == 1.0][feature], y=X[X[color] != 0.0][target], mode="markers", name="Yes", hovertext=X["Institution Name"]
    ))
    
    
    fig2.add_trace(go.Scatter(
        x=sample[feature], y=sample_y[target], mode="markers", name="Sample Universities", hovertext="Duquesne"
    ))
    
    fig2.update_layout(
        title="HBCU " + target +  " Vs. Price",
        xaxis={
            "title": feature,
            "range": [df[feature].min() - df[feature].std() / 10, df[feature].max() + df[feature].std() / 10]
        },
        yaxis={
            "title": "Graduation Rate",
            "range": [-5,105]
        },
        legend={
            'orientation': 'v',
            'title': color,
            'x': 0.5,
            'y': 1.1,
            'bgcolor': "rgba(176,196,222,0.7)",
            'bordercolor': "black",
            'borderwidth': 2,
            'itemclick': "toggle",
        },
        font=dict(
            family="Courier New, monospace",
            size=14),
        height=700
    )
    
    fig2.update_traces(marker=dict(size=12,
                                  line=dict(width=2,
                                  color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    

    display(fig2.show())
    

ui = wg.HBox([feature_picker, target_picker, color_picker])  

func_params = {
    'feature': feature_picker,
    'target': target_picker,
    'color': color_picker
}


output = wg.interactive_output(fe_plot, func_params)   


display(ui, output)   

Output()

In [29]:
set(df.index) - set(X_df.index)

{130943, 141060, 199111}

<h1><center>Human Resources (Total Instructional Staff)</center></h1>

In [149]:
range_ = [12, 26]
degree = wg.IntSlider(value=1, min=1, max=3)

feature_picker = wg.Dropdown(
    options=X.columns,
    value=X.filter(regex="Grand.*instructional_staff").columns[0],
    description='Feature:',
    disabled=False,
)
target_picker = wg.Dropdown(
    options=grad_rates.columns,
    value=grad_rates.columns[0],
    description='Feature:',
    disabled=False,
)
color_picker = wg.Dropdown(
    options=X.filter(regex="Sector|Historically|State|Basic|Size and Setting|category").columns,
    value=X.filter(regex="Historically").columns[0],
    description='Color:',
    disabled=False,
)


def fe_plot(feature, target, color):
    
    fig2 = go.Figure()
    
    fig2.add_trace(go.Scatter(
        x=X[X[color] == 0.0][feature], y=X[X[color] == 0.0][target], mode="markers", name="No", hovertext=X["Institution Name"]
    ))
    
    fig2.add_trace(go.Scatter(
        x=X[X[color] == 1.0][feature], y=X[X[color] != 0.0][target], mode="markers", name="Yes", hovertext=X["Institution Name"]
    ))
    
    
    fig2.add_trace(go.Scatter(
        x=x_holdout[feature], y=y_holdout, mode="markers", name="Sample Universities", hovertext=x_holdout.index, line=dict(color="rgba(20,220,20,1)")
    ))
    
    fig2.update_layout(
        title="Grand Total Instructional Staff",
        xaxis={
            "title": feature,
            "range": [df[feature].min() - df[feature].std() / 10, df[feature].max() + df[feature].std() / 10]
        },
        yaxis={
            "title": "Graduation Rate",
            "range": [-5,105]
        },
        legend={
            'orientation': 'v',
            'title': color,
            'x': 0.5,
            'y': 1.1,
            'bgcolor': "rgba(176,196,222,0.7)",
            'bordercolor': "black",
            'borderwidth': 2,
            'itemclick': "toggle",
        },
        font=dict(
            family="Courier New, monospace",
            size=14),
        height=700
    )

    fig2.update_traces(marker=dict(size=12,
                                  line=dict(width=2,
                                  color='DarkSlateGrey')),
                      selector=dict(mode='markers'))

    display(fig2.show())
    

ui = wg.HBox([feature_picker, target_picker, color_picker])  

func_params = {
    'feature': feature_picker,
    'target': target_picker,
    'color': color_picker
}


output = wg.interactive_output(fe_plot, func_params)   


display(ui, output)   

Output()

In [31]:
cc_ss = df.filter(regex="Size and Setting").columns[0]

In [32]:
highly = 'highly residential'
nonres = 'primarily nonresidential'
res = 'primarily residential'

In [33]:
res_level_map = df[cc_ss].map({
    'Two-year, very small': np.nan,
    'Two-year, small': np.nan,
    'Two-year, medium': np.nan,
    'Two-year, large': np.nan,
    'Two-year, very large': np.nan,
    
    'Four-year, very small, primarily nonresidential': nonres,
    'Four-year, very small, primarily residential': res,
    'Four-year, very small, highly residential': highly,
    
    'Four-year, small, primarily nonresidential': nonres,
    'Four-year, small, primarily residential': res,
    'Four-year, small, highly residential': highly,
    
    'Four-year, medium, primarily nonresidential': nonres,
    'Four-year, medium, primarily residential': res,
    'Four-year, medium, highly residential': highly,
    
    
    'Four-year, large, primarily nonresidential': nonres,
    'Four-year, large, primarily residential': res,
    'Four-year, large, highly residential': highly,
    
    'Not applicable, not in Carnegie universe (not accredited or nondegree-granting)': np.nan,
    'Exclusively graduate/professional': np.nan,
    'isMissing': np.nan
})

In [34]:
X["Residential_Level"] = res_level_map

In [35]:
X["Log_Core_Revenues"] = np.log(X["Core_Revenues"])
X["Log_Revenue_Level"] = pd.cut(X["Log_Core_Revenues"], bins=3, labels=[0, 1, 2])

In [36]:
labels = ['<=15.6', '15.6 < Rev <= 19.1', ' 19.1 < Rev <= 22.8']

In [37]:
rev_level = X["Log_Revenue_Level"].dropna().unique()
res_level = X["Residential_Level"].dropna().unique()

In [38]:
X[X["Log_Revenue_Level"] == rev_level[1]]["Log_Revenue_Level"][X["Residential_Level"] == res_level[1]]

UnitID
219505    0
444398    0
176789    0
224244    0
194569    0
202903    0
198677    0
436614    0
421832    0
192624    0
230977    0
208725    0
439862    0
194666    0
484871    0
207157    0
123280    0
181543    0
137953    0
137962    0
234137    0
197601    0
488101    0
Name: Log_Revenue_Level, dtype: category
Categories (3, int64): [0 < 1 < 2]

<h1><center>Residential Level</center></h1>

In [43]:
# Creating one series that contains residential Level (nonresidential, residential, highly residential)
highly_series = X_fe.filter(regex="highly").sum(axis=1).replace({0: np.nan, 1: highly})
non_series = X_fe.filter(regex="primarily nonresidential").sum(axis=1).replace({0: np.nan, 1: nonres})
res_series = X_fe.filter(regex="primarily residential").sum(axis=1).replace({0: np.nan, 1: res})

residential_series = pd.concat([highly_series, non_series, res_series]).dropna()
residential_series = pd.DataFrame(residential_series).reset_index()

In [45]:
# merging that series back on to X_features
X_fe = X_fe.reset_index().merge(residential_series, on="UnitID", how="outer")

X_fe.rename({0: "Residential_Level"}, axis=1, inplace=True)
# Fill missing with "Missing"
X_fe.Residential_Level.fillna("Missing", inplace=True)
# Merge y onto features df
X_fe = X_fe.merge(y.reset_index(), on="UnitID", how="outer")
# Reeset the index back to "UnitID"
X_fe.set_index("UnitID", inplace=True)

In [49]:
import plotly.io as pio

# naming a layout theme for future reference
pio.templates["google"] = go.layout.Template(
    layout_colorway=['#4285F4', '#DB4437', '#F4B400', '#0F9D58',
                     '#185ABC', '#B31412', '#EA8600', '#137333',
                     '#d2e3fc', '#ceead6']
)
# setting Google color palette as default

# pio.templates.default = "google"

In [51]:
# Elastic Net Model used for predictions
elastic_net = model_log[-1]['model']

In [52]:

enet_pred = elastic_net.predict(x_test)

In [53]:

coefs = elastic_net['classifier'].coef_
coefs_dict = dict(zip(X_df.columns, coefs))
sorted_coefs = {i: v for i, v in sorted(coefs_dict.items(), key=lambda x: np.abs(x[1]))}

In [54]:
coefs_df = pd.DataFrame(sorted_coefs, index=[0])

In [55]:
sorted_df = coefs_df.iloc[:, -10:].transpose().sort_values(by=0).transpose()

<h1><center>Elastic Net Coefficients</center></h1>

In [222]:
sorted_df

,Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Percent of total enrollment that are Black or African American (DRVEF2013_RV),Log_Tuition_And_Fees_As_Dollar_Amount,Government_Grants,Student_Service_Expenses,ACT Composite 75th percentile score (IC2012_RV),"Institution size category (HD2018)_Under 1,000",Total price for in-district students living off campus (with family) 2012-13 (DRVIC2012_RV),Log_Core_Revenues,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819)
0,-4.424078,-3.05555,-2.957731,-2.63252,-2.618674,2.564262,2.570295,2.791732,3.215251,3.548572


In [223]:
# plot first 5 and last 5 on plotly graph
fig = go.Figure()
fig.add_trace(go.Bar(
    y=sorted_df.iloc[0,:], x=sorted_df.columns, orientation='v', text=np.round(sorted_df.iloc[0], 2)
))

fig.update_layout(
    barmode='group',
    title="Elastic Net Coefficients",
    title_x = 0.5,
    height=700,
    font={
        "size": 16
    },
    xaxis={
        "tickvals": sorted_df.columns.to_list(),
        "ticktext":["% awarded grant aid", "% Enrollment Black", "($) Log Tuition + Fees", "(%) Government Grants",
                    "(%) Student Service Expenses", "ACT Composite", "Size_Under_1K", "Price (in-district with family)", "$ Log Core Rev", "% Awarded any type of aid"],
        "tickfont": {
            'size': 12
        },
        "title": "Feature Name"
        
        
        
    }
)


fig.show()

In [178]:
aid = sorted_df.columns[0]
aid

'Percent of full-time first-time undergraduates awarded federal  state  local or institutional grant aid (SFA1819)'

In [219]:
def interperate(col, coef):
    scaler = StandardScaler()
    transformed = scaler.fit_transform(x_train[col].to_numpy().reshape(-1,1))
    print(np.mean(transformed), np.std(transformed))
    observation = np.array([[coef]])                          
    observation = scaler.inverse_transform(observation)
    
    return observation[0][0] / coef
    

In [224]:
coef_interpretation = interperate(aid, -4.42)

3.808918992175537e-16 0.9999999999999999


>{coef_interpretation}
> "This is a quote from 
>
> <a src="https://www.youtube.com/watch?v=-F4WS8o-G2A">Robert Campbell</a>

```python
print("this is python code")
```

In [39]:

fig = go.Figure()

fig.add_trace(
    go.Violin(x=X["Residential_Level"][X["Residential_Level"] == nonres], y=X[total][X["Residential_Level"] == nonres],
             name=nonres)
)
fig.add_trace(
    go.Violin(x=X["Residential_Level"][X["Residential_Level"] == res], y=X[total][X["Residential_Level"] == res],
             name=res)
)
fig.add_trace(
    go.Violin(x=X["Residential_Level"][X["Residential_Level"] == highly], y=X[total][X["Residential_Level"] == highly],
             name=highly)
)


fig.update_traces(box_visible=True, meanline_visible=True)

fig.update_layout(
    height=600,
    width = 1200,
    title="Graduation Rate by Residential Level and Controlling for Revenue Level",
    legend={
        "title": "Legend",
        "orientation": "v",   
        "itemclick": "toggle"
    },
    font={
       "family": "Courier New, Monospace",
        "size": 18
    }
)

fig.update_yaxes()


In [166]:
fig = px.scatter(X_fe, x="Log_Core_Revenues", y=total, color="Residential_Level", trendline='ols')

tr_line=[]
scatter = []
for  k, trace  in enumerate(fig.data):
    if trace.mode is not None and trace.mode == 'lines':
        tr_line.append(k)
    else:
        scatter.append(k)



for id_ in tr_line:
    fig.data[id_].update(line_width=4)
for id_ in scatter:
    fig.data[id_].update(opacity=0.5)
    
    
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(
    marker={
       "line": {
           "width": 2,
       },
       "size": 10,
    },
    showlegend=True
    
)
fig.update_layout(
    title="Impact of Residential Level On Slope",
    font={
        'size': 16
    },
    xaxis={
        "title": "Log Revenues",
        "range": [13,22.5]
    },
    yaxis={
        "title": "Graduation Rate",
        "range": [15,100]
    },
    legend={
        'orientation': 'v',
        'title': "Residential Level",
        'x': 0.005,
        'y': 0.99,
        'bgcolor': "rgba(176,196,222,0.7)",
        'bordercolor': "black",
        'borderwidth': 2,
        'itemclick': "toggle",
        'font': {
            'size': 16
        }
    },
    height= 600,
    width=800
)



fig.show()

In [154]:
name = df.loc[my_id, "Institution Name"]

<ul><li><h3>One Coefficient for the effect of Revenue on Graduation Rate</h3></li></ul>

<h1><center>XGBoost Feature Importances</center></h1>

In [86]:
importances = model_log[-2]['model']['classifier'].feature_importances_

In [87]:
sorted_importances = sorted(list(zip(x_train.columns, importances)), key=lambda x: x[1])

In [89]:
# plot first 5 and last 5 on plotly graph
fig = go.Figure()
fig.add_trace(go.Bar(
    y=[i[1] for i in sorted_importances[-10:]], x=[i[0] for i in sorted_importances[-10:]], orientation='v', text=[np.round(i[1], 3) for i in sorted_importances[-10:]]
))

fig.update_layout(
    barmode='group',
    title="XGB Feature Importances",
    title_x=0.5,
    height=800,
    font={
        "size": 16
    },
    xaxis={
        "tickvals": [i[0] for i in sorted_importances[-10:]],
        "ticktext":["Government Grants", "($) Private Gifts ", "Is_4year_Private_for-profit", "(%) Grand Total Undergraduate  Age 25 and over", "Total Men Undergraduate  Age 25 and over", 
                    "Off Campus Price (w/o family)", "SAT Reading", "ACT Composite", "SAT MMath", 'price*percentage of women Under 25'],
        "tickfont": {
            'size': 12
        },
        
        
        
    }
)
fig.update_xaxes()

fig.show()

<h1><center>Interpretation of XGB</center></h1>

<h1><center>Model Performance</center></h1>

In [90]:
model = model_log[-2]["model"]
y_pred = model.predict(x_test)

holdout_pred = model.predict(x_holdout)

In [91]:
enet = model_log[-1]["model"]
enet_pred = enet.predict(x_test)

In [168]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=y_pred, y=y_test, name="XGB(pred, obs)", mode="markers")
)
fig.add_trace(
    go.Scatter(x=enet_pred, y=y_test, name="ElasticNet(pred, obs)", mode="markers")
)
fig.add_trace(
    go.Scatter(x=holdout_pred, y=y_holdout, name="(predicted, observed)", mode="markers", hovertext=x_holdout.index)
)
fig.add_trace(
    go.Scatter(x=np.linspace(0,100, 100), y=np.linspace(0,100,100), name="100% Accuracy", line_width=4)
)
fig.update_layout(
    title="Model Performance",
    xaxis={
        "title": "Predicted Value",
        "range": [-2, 102]
    },
    yaxis={
        "title": "Observed Value",
        "range": [-2, 102]
    },
    
    font=dict(
        family="Courier New, monospace",
        size=14),
    legend={
        'orientation': 'v',
        'title': "Legend",
        'x': 0.01,
        'y': 0.9,
        'bgcolor': "rgba(176,196,222,0.7)",
        'bordercolor': "black",
        'borderwidth': 2,
        'itemclick': "toggle",
    },
    height=700
)

fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                              color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()